In [67]:
import xarray as xr
import numpy as np
import os
from math import pi
import pandas as pd
import cfgrib

# Features

In [3]:
data = xr.open_dataset('Z:\\data\\PERIFEREIA_Attikis\\0_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\prediction.grib', engine='cfgrib')

In [4]:
data

<xarray.Dataset>
Dimensions:     (time: 93, step: 24, latitude: 3, longitude: 10)
Coordinates:
    number      int32 ...
  * time        (time) datetime64[ns] 2022-04-30 2022-05-01 ... 2022-07-31
  * step        (step) timedelta64[ns] 01:00:00 02:00:00 ... 1 days 00:00:00
    surface     float64 ...
  * latitude    (latitude) float64 38.22 38.12 38.02
  * longitude   (longitude) float64 23.09 23.19 23.29 ... 23.79 23.89 23.99
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    u10         (time, step, latitude, longitude) float32 ...
    v10         (time, step, latitude, longitude) float32 ...
    t2m         (time, step, latitude, longitude) float32 ...
    tp          (time, step, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-09-26T13:10 GRIB to CDM+CF via cfgrib-0.9.1...

t2m -> Θερμοκρασία στα 2 m ανά ώρα -> Μέγιστη, ελάχιστη και μέση θερμοκρασία ημέρας

u10,v10 ->Συνιστώσες του ανέμου -> max_velocity, direction, dominant direction, max_velocity in the dominant direction, 

In [21]:
def make_temp_wind(data,date):
    data['max_temp'] = data.t2m.max(dim='step')
    data['min_temp'] = data.t2m.min(dim='step')
    data['mean_temp'] = data.t2m.mean(dim='step')
    #ταχύτητα ανέμου - Πυθαγόρειο
    data['wind'] = np.sqrt(data.u10**2+data.v10**2)
    #Κατεύθυνση ανέμου
    data['dir'] = 180 + (180 / pi) * np.arctan2(data.u10, data.v10)
    data['dir_cat'] = (data.dir>0).astype(int) + (data.dir>22.5).astype(int) + (data.dir>67.5).astype(int) + \
    (data.dir>112.5).astype(int) + (data.dir>157.5).astype(int) + \
    (data.dir>202.5).astype(int) + (data.dir>247.5).astype(int) + \
    (data.dir>292.5).astype(int) - 7*(data.dir>337.5).astype(int)
    #To be checked
    data['dom_dir'] = xr.DataArray(np.apply_along_axis(majority,0,data.dir_cat),dims=['latitude','longitude'])
    data['masked_velocity'] = data.wind.where(data.dir_cat==data.dom_dir , drop=False)
    data['dom_vel'] = data['masked_velocity'].max(dim='step')
    data['res_max'] = data['wind'].max(dim='step')
    data['dir_max'] = data.wind.where(data.wind==data.res_max,drop=False).max(dim='step')
    out = data[['dom_dir','dom_vel','res_max','dir_max','max_temp','min_temp','mean_temp']]
    out = out.assign_coords({'time':date})
    time_name = pd.to_datetime(str(date)).strftime('%Y%m%d')
    out.to_netcdf('Z:\\data\\PERIFEREIA_Attikis\\0_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\temp_wind_2022\\'+time_name+"_temp_wind.nc")

In [81]:
def make_temp_wind_prec(data,date):
    data['max_temp'] = data.t2m.max(dim='step')
    data['min_temp'] = data.t2m.min(dim='step')
    data['mean_temp'] = data.t2m.mean(dim='step')
    #ταχύτητα ανέμου - Πυθαγόρειο
    data['wind'] = np.sqrt(data.u10**2+data.v10**2)
    #Κατεύθυνση ανέμου
    data['dir'] = 180 + (180 / pi) * np.arctan2(data.u10, data.v10)
    data['dir_cat'] = (data.dir>0).astype(int) + (data.dir>22.5).astype(int) + (data.dir>67.5).astype(int) + \
    (data.dir>112.5).astype(int) + (data.dir>157.5).astype(int) + \
    (data.dir>202.5).astype(int) + (data.dir>247.5).astype(int) + \
    (data.dir>292.5).astype(int) - 7*(data.dir>337.5).astype(int)
    #To be checked
    data['dom_dir'] = xr.DataArray(np.apply_along_axis(majority,0,data.dir_cat),dims=['latitude','longitude'])
    data['masked_velocity'] = data.wind.where(data.dir_cat==data.dom_dir , drop=False)
    data['dom_vel'] = data['masked_velocity'].max(dim='step')
    data['res_max'] = data['wind'].max(dim='step')
    data['dir_max'] = data.wind.where(data.wind==data.res_max,drop=False).max(dim='step')
    out = data[['dom_dir','dom_vel','res_max','dir_max','max_temp','min_temp','mean_temp']]
    out = out.assign_coords({'time':date})
        for i in range(6,data.time.values.shape[0]):
        data = data1.isel(time=list(np.arange(i-6,i+1,1)),step=23)
        data['rain_7_days'] =  data1['tp'].sum(dim='time')
        date = data1.time.values[-1]
        out = data1[['rain_7_days']]
        out = out.assign_coords({'time':date})
    time_name = pd.to_datetime(str(date)).strftime('%Y%m%d')
    out.to_netcdf('Z:\\data\\PERIFEREIA_Attikis\\0_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\2022\\'+time_name+"_temp_wind_prec.nc")

IndentationError: unexpected indent (2706855203.py, line 21)

## 

# Rain

In [23]:
data = xr.open_dataset('Z:\\data\\PERIFEREIA_Attikis\\0_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\prediction_august.grib', engine='cfgrib')

Ignoring index file 'Z:\\data\\PERIFEREIA_Attikis\\0_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\prediction_august.grib.923a8.idx' older than GRIB file


In [24]:
data

<xarray.Dataset>
Dimensions:     (latitude: 3, longitude: 10)
Coordinates:
    number      int32 ...
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 38.22 38.12 38.02
  * longitude   (longitude) float64 23.09 23.19 23.29 ... 23.79 23.89 23.99
    valid_time  datetime64[ns] ...
Data variables:
    u10         (latitude, longitude) float32 ...
    v10         (latitude, longitude) float32 ...
    t2m         (latitude, longitude) float32 ...
    tp          (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-09-23T15:38 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
for i in range(6,data.time.values.shape[0]):
    data1 = data.isel(time=list(np.arange(i-6,i+1,1)),step=23)
    data1['rain_7_days'] =  data1['tp'].sum(dim='time')
    date = data1.time.values[-1]
    out = data1[['rain_7_days']]
    out = out.assign_coords({'time':date})
    time_name = pd.to_datetime(str(date)).strftime('%Y%m%d')
    out.to_netcdf('Z:\\data\\PERIFEREIA_Attikis\\0_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\precipitation_2022\\'+time_name+"_prec.nc")

IndexError: tuple index out of range

In [14]:
#data.time[0]

IndexError: too many indices

# Features to shapefile

In [49]:
import geopandas as gpd

In [50]:
points = gpd.read_file('Z:\\data\\PERIFEREIA_Attikis\\p3\\grid_wgs84.shp')

In [51]:
#Explode because it is multipoint
points = points.explode(index_parts=False)

In [52]:
points

,,id,geometry
0,0,543,POINT (23.39644 38.21008)
1,0,544,POINT (23.39758 38.21008)
2,0,545,POINT (23.39872 38.21009)
3,0,546,POINT (23.39986 38.21010)
4,0,547,POINT (23.40100 38.21010)
...,...,...,...
47446,0,69345,POINT (23.40020 38.02985)
47447,0,69685,POINT (23.39565 38.02892)
47448,0,69686,POINT (23.39679 38.02893)
47449,0,69687,POINT (23.39792 38.02893)


In [47]:
datalist = ['20060716', '20070628', '20080715', '20090820','20090822', '20150613', '20160613', '20160625','20210602', '20210816', '20220705', '20220805']

In [44]:
new_features = ['dom_dir','dom_vel','res_max','dir_max','max_temp','min_temp','mean_temp','rain_7_days']

In [ ]:
#os.path.join('/mnt/storageapplications/data/PERIFEREIA_Attikis/p3','20210906'+'_temp_wind.nc')

In [ ]:
Z:\\data\\PERIFEREIA_Attikis\\0_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\precipitation_2022\\'+time_name+"_prec.nc"

In [16]:
save_file = '/mnt/storageapplications/data/PERIFEREIA_Attikis/p3'

In [34]:
save_file1 = 'Z:\\data\\PERIFEREIA_Attikis\0_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\precipitation_2022'

In [35]:
save_file2 = 'Z:\\data\\PERIFEREIA_Attikis\0_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\temp_wind_2022'

In [36]:
save_file3 = 'Z:\\data\\PERIFEREIA_Attikis\\0_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\era5\\2022'

In [37]:
import datetime
import pandas as pd

In [39]:
start = datetime.datetime.strptime("20220506", "%Y%m%d")
end = datetime.datetime.strptime("20220731", "%Y%m%d")
date_generated = pd.date_range(start, end)
print(date_generated.strftime("%Y%m%d"))

Index(['20220506', '20220507', '20220508', '20220509', '20220510', '20220511',
       '20220512', '20220513', '20220514', '20220515', '20220516', '20220517',
       '20220518', '20220519', '20220520', '20220521', '20220522', '20220523',
       '20220524', '20220525', '20220526', '20220527', '20220528', '20220529',
       '20220530', '20220531', '20220601', '20220602', '20220603', '20220604',
       '20220605', '20220606', '20220607', '20220608', '20220609', '20220610',
       '20220611', '20220612', '20220613', '20220614', '20220615', '20220616',
       '20220617', '20220618', '20220619', '20220620', '20220621', '20220622',
       '20220623', '20220624', '20220625', '20220626', '20220627', '20220628',
       '20220629', '20220630', '20220701', '20220702', '20220703', '20220704',
       '20220705', '20220706', '20220707', '20220708', '20220709', '20220710',
       '20220711', '20220712', '20220713', '20220714', '20220715', '20220716',
       '20220717', '20220718', '20220719', '20220720

In [61]:
datalist = ['20220506', '20220507', '20220508', '20220509', '20220510', '20220511',
       '20220512', '20220513', '20220514', '20220515', '20220516', '20220517',
       '20220518', '20220519', '20220520', '20220521', '20220522', '20220523',
       '20220524', '20220525', '20220526', '20220527', '20220528', '20220529',
       '20220530', '20220531', '20220601', '20220602', '20220603', '20220604',
       '20220605', '20220606', '20220607', '20220608', '20220609', '20220610',
       '20220611', '20220612', '20220613', '20220614', '20220615', '20220616',
       '20220617', '20220618', '20220619', '20220620', '20220621', '20220622',
       '20220623', '20220624', '20220625', '20220626', '20220627', '20220628',
       '20220629', '20220630', '20220701', '20220702', '20220703', '20220704',
       '20220705', '20220706', '20220707', '20220708', '20220709', '20220710',
       '20220711', '20220712', '20220713', '20220714', '20220715', '20220716',
       '20220717', '20220718', '20220719', '20220720', '20220721', '20220722',
       '20220723', '20220724', '20220725', '20220726', '20220727', '20220728',
       '20220729', '20220730', '20220731']

In [62]:
list_of_files = [os.path.join(save_file1,date+'_temp_wind.nc'),os.path.join(save_file2,date+'_prec.nc')]

['Z:\\data\\PERIFEREIA_Attikis\x00_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\precipitation_2022\\20060716_temp_wind.nc',
 'Z:\\data\\PERIFEREIA_Attikis\x00_Data_Management\\Phases\\Phase_3\\01_Hazard\\Fire\\Input_Data\\dataset\\temp_wind_2022\\20060716_prec.nc']

In [69]:
!pip install netCDF4 

In [71]:
!conda install -c netcdf4 -y


CondaValueError: too few arguments, must supply command line package specs or --file



In [72]:
!conda install -c scipy -y


CondaValueError: too few arguments, must supply command line package specs or --file



In [73]:
!conda install -c pydap -y


CondaValueError: too few arguments, must supply command line package specs or --file



In [74]:
!conda install -c zarr -y


CondaValueError: too few arguments, must supply command line package specs or --file



In [75]:
!conda install -c fsspec -y


CondaValueError: too few arguments, must supply command line package specs or --file



In [76]:
!conda install -c cftime -y


CondaValueError: too few arguments, must supply command line package specs or --file



In [77]:
!conda install -c rasterio -y


CondaValueError: too few arguments, must supply command line package specs or --file



In [78]:
!conda install -c  -ypooch -y

usage: conda-script.py install [-h] [--revision REVISION]
                               [-n ENVIRONMENT | -p PATH] [-c CHANNEL]
                               [--use-local] [--override-channels]
                               [--repodata-fn REPODATA_FNS]
                               [--strict-channel-priority]
                               [--no-channel-priority]
                               [--no-deps | --only-deps] [--no-pin] [--copy]
                               [--no-shortcuts] [-C] [-k] [--offline] [-d]
                               [--json] [-q] [-v] [-y] [--download-only]
                               [--show-channel-urls] [--file FILE]
                               [--experimental-solver {classic,libmamba,libmamba-draft}]
                               [--force-reinstall]
                               [--freeze-installed | --update-deps | -S | --update-all | --update-specs]
                               [-m] [--clobber] [--dev]
                               [packa

In [70]:
meteo_date = xr.open_mfdataset(list_of_files)

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'cfgrib']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [63]:
for date in datalist:
    print(date)
    list_of_files = [os.path.join(save_file1,date+'_temp_wind.nc'),os.path.join(save_file2,date+'_prec.nc')]
    meteo_date = xr.open_mfdataset(list_of_files)
    for feat in new_features:
        print(feat)
        points[feat] = points.apply(lambda row: new_features(row,feat),axis=1)
        print(points)
    points.to_file(os.path.join(save_file3,date+'_temp_wind_prec.shp'))

20220506


ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'cfgrib']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [56]:
meteo_date = xr.open_mfdataset(list_of_files)

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'cfgrib']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [20]:
meteo_date

<xarray.Dataset>
Dimensions:      (latitude: 3, longitude: 8)
Coordinates:
  * latitude     (latitude) float64 38.22 38.12 38.02
  * longitude    (longitude) float64 23.09 23.19 23.29 ... 23.59 23.69 23.79
    number       int64 0
    time         datetime64[ns] 2006-07-16
    surface      float64 0.0
    step         timedelta64[ns] ...
Data variables:
    dom_dir      (latitude, longitude) int64 dask.array<chunksize=(3, 8), meta=np.ndarray>
    dom_vel      (latitude, longitude) float32 dask.array<chunksize=(3, 8), meta=np.ndarray>
    res_max      (latitude, longitude) float32 dask.array<chunksize=(3, 8), meta=np.ndarray>
    dir_max      (latitude, longitude) float32 dask.array<chunksize=(3, 8), meta=np.ndarray>
    max_temp     (latitude, longitude) float32 dask.array<chunksize=(3, 8), meta=np.ndarray>
    min_temp     (latitude, longitude) float32 dask.array<chunksize=(3, 8), meta=np.ndarray>
    mean_temp    (latitude, longitude) float32 dask.array<chunksize=(3, 8), meta=np.ndarray>
    rain_7_days  (latitude, longitude) float32 dask.array<chunksize=(3, 8), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-09-20T15:06 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
import nctoolkit as nc
# read in the files
ds = nc.open_data("infile1.nc")
ds2 = nc.open_data("infile2.nc")
# remove the 1st netcdf files variables from the second's
ds2.drop(ds.variables)
# merge the files
ds.append(ds2)
ds.merge()
# save the files as a netcdf file
ds.to_nc("merged.nc")

In [40]:
def features(row,feature):
    lon = row['geometry'].x
    lat = row['geometry'].y
    sel = meteo_date.sel(latitude= lat,longitude=lon, method="nearest")
    value = float(sel[feature].values)
#     dom_vel = float(sel.dom_vel.data)
#     res_max = float(sel.res_max.data)
#     dir_max = float(sel.dir_max.data)
#     max_temp = float(sel.max_temp.data)
#     min_temp = float(sel.min_temp.data)
#     mean_temp = float(sel.mean_temp.data)
#    return dom_dir,dom_vel,res_max,dir_max,max_temp,min_temp,mean_temp
    return value

In [ ]:
str(data.time.data[0])

In [ ]:
#for onedate in out.time:
    #for feat in new_features:
       # points[feat] = points.apply(lambda row: features(row,feat),axis=1)
       # point.to_file(''+str(data.time.data[0])+'temp_wind.shp')

In [ ]:
#point.to_file('/path/for/save/filename.shp')

# Join with static csvs

https://towardsdatascience.com/left-join-with-pandas-data-frames-in-python-c29c85089ba4

In [ ]:
import pandas as pd

In [ ]:
dem = pd.read_csv('')

In [ ]:
save_path = '/mnt/storageapplications/data/PERIFEREIA_Attikis/p3'
for date in dateslist:
    temp_file = date+'_temp_wind.nc'
    temp = xr.open_dataset(os.path.join(save_path,temp_file))
    prec_file = date+'_prec.nc'
    prec = xr.open_dataset(os.path.join(save_path,prec_file))

# Check frequency

In [27]:
data1 = data.isel(time=1) 

In [28]:
data.isel(time=1)

<xarray.Dataset>
Dimensions:     (step: 24, latitude: 3, longitude: 10)
Coordinates:
    number      int32 0
    time        datetime64[ns] 2022-05-01
  * step        (step) timedelta64[ns] 01:00:00 02:00:00 ... 1 days 00:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 38.22 38.12 38.02
  * longitude   (longitude) float64 23.09 23.19 23.29 ... 23.79 23.89 23.99
    valid_time  (step) datetime64[ns] 2022-05-01T01:00:00 ... 2022-05-02
Data variables:
    u10         (step, latitude, longitude) float32 0.2903 0.4186 ... 0.2004
    v10         (step, latitude, longitude) float32 -1.444 -1.444 ... -3.072
    t2m         (step, latitude, longitude) float32 284.6 284.4 ... 286.8 287.2
    tp          (step, latitude, longitude) float32 0.0 0.0 ... 4.466e-05
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-09-26T13:10 GRIB to CDM+CF via cfgrib-0.9.1...

In [29]:
def majority(x):
    counts = np.array(np.unique(x, return_counts=True)).T
    print(np.bincount(x).argmax())
    print(counts)
    return np.bincount(x).argmax() #για να επιστρέφει από τα counts το μέγιστο
def dom_vel(x):
    return max(x)

In [30]:
data1['max_temp'] = data1.t2m.max(dim='step')
data1['min_temp'] = data1.t2m.min(dim='step')
data1['mean_temp'] = data1.t2m.mean(dim='step')
#ταχύτητα ανέμου - Πυθαγόρειο
data1['wind'] = np.sqrt(data1.u10**2+data1.v10**2)
#Κατεύθυνση ανέμου
data1['dir'] = 180 + (180 / pi) * np.arctan2(data1.u10, data1.v10)
data1['dir_cat'] = (data1.dir>0).astype(int) + (data1.dir>22.5).astype(int) + (data1.dir>67.5).astype(int) + \
(data1.dir>112.5).astype(int) + (data1.dir>157.5).astype(int) + \
(data1.dir>202.5).astype(int) + (data1.dir>247.5).astype(int) + \
(data1.dir>292.5).astype(int) - 7*(data1.dir>337.5).astype(int)
#To be checked
data1['dom_dir'] = xr.DataArray(np.apply_along_axis(majority,0,data1.dir_cat),dims=['latitude','longitude'])
data1['masked_velocity'] = data1.wind.where(data1.dir_cat==data1.dom_dir , drop=False)
data1['dom_vel'] = data1['masked_velocity'].max(dim='step')
data1['res_max'] = data1['wind'].max(dim='step')
data1['dir_max'] = data1.wind.where(data1.wind==data1.res_max,drop=False).max(dim='step')

1
[[ 1 19]
 [ 2  4]
 [ 8  1]]
1
[[ 1 18]
 [ 2  5]
 [ 8  1]]
1
[[ 1 17]
 [ 2  6]
 [ 8  1]]
1
[[ 1 14]
 [ 2  5]
 [ 8  5]]
1
[[ 1 13]
 [ 2  3]
 [ 8  8]]
1
[[ 1 12]
 [ 2  3]
 [ 8  9]]
1
[[ 1 12]
 [ 2  5]
 [ 8  7]]
1
[[ 1 13]
 [ 2  5]
 [ 8  6]]
1
[[ 1 16]
 [ 2  5]
 [ 8  3]]
1
[[ 1 18]
 [ 2  4]
 [ 8  2]]
1
[[ 1 19]
 [ 2  4]
 [ 8  1]]
0
[[ 0 24]]
1
[[ 1 16]
 [ 2  7]
 [ 8  1]]
1
[[ 1 16]
 [ 2  6]
 [ 8  2]]
1
[[ 1 15]
 [ 2  6]
 [ 8  3]]
1
[[ 1 15]
 [ 2  5]
 [ 8  4]]
1
[[ 1 15]
 [ 2  5]
 [ 8  4]]
1
[[ 1 17]
 [ 2  5]
 [ 8  2]]
1
[[ 1 19]
 [ 2  5]]
1
[[ 1 19]
 [ 2  5]]
1
[[ 1 15]
 [ 2  7]
 [ 8  2]]
1
[[ 1 15]
 [ 2  7]
 [ 8  2]]
1
[[ 1 14]
 [ 2  8]
 [ 8  2]]
1
[[ 1 13]
 [ 2  8]
 [ 8  3]]
1
[[ 1 14]
 [ 2  7]
 [ 8  3]]
1
[[ 1 14]
 [ 2  7]
 [ 8  3]]
1
[[ 1 16]
 [ 2  5]
 [ 8  3]]
1
[[ 1 17]
 [ 2  6]
 [ 8  1]]
1
[[ 1 18]
 [ 2  6]]
1
[[ 1 18]
 [ 2  6]]


In [31]:
data1

<xarray.Dataset>
Dimensions:          (step: 24, latitude: 3, longitude: 10)
Coordinates:
  * step             (step) timedelta64[ns] 01:00:00 ... 1 days 00:00:00
  * latitude         (latitude) float64 38.22 38.12 38.02
  * longitude        (longitude) float64 23.09 23.19 23.29 ... 23.79 23.89 23.99
    number           int32 0
    time             datetime64[ns] 2022-05-01
    surface          float64 0.0
    valid_time       (step) datetime64[ns] 2022-05-01T01:00:00 ... 2022-05-02
Data variables: (12/15)
    u10              (step, latitude, longitude) float32 0.2903 ... 0.2004
    v10              (step, latitude, longitude) float32 -1.444 ... -3.072
    t2m              (step, latitude, longitude) float32 284.6 284.4 ... 287.2
    tp               (step, latitude, longitude) float32 0.0 0.0 ... 4.466e-05
    max_temp         (latitude, longitude) float32 294.0 294.1 ... 295.0 295.3
    min_temp         (latitude, longitude) float32 284.1 284.1 ... 285.5 285.8
    ...               ...
    dir_cat          (step, latitude, longitude) int32 1 1 1 8 8 8 ... 1 1 1 1 1
    dom_dir          (latitude, longitude) int64 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    masked_velocity  (step, latitude, longitude) float32 1.473 1.503 ... 3.079
    dom_vel          (latitude, longitude) float32 3.875 3.607 ... 4.391 4.617
    res_max          (latitude, longitude) float32 4.086 3.735 ... 4.47 4.711
    dir_max          (latitude, longitude) float32 4.086 3.735 ... 4.47 4.711
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-09-26T13:10 GRIB to CDM+CF via cfgrib-0.9.1...

In [17]:
#data1['dom_dir'] = xr.DataArray(np.apply_along_axis(majority,0,data1.dir_cat),dims=['latitude','longitude'])

In [32]:
data1.dir_cat[:,0,0] #για να ελέγξεις αν βγάζει σωστό αποτέλεσμα το majority

<xarray.DataArray 'dir_cat' (step: 24)>
array([1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1,
       1, 1])
Coordinates:
  * step        (step) timedelta64[ns] 01:00:00 02:00:00 ... 1 days 00:00:00
    latitude    float64 38.22
    longitude   float64 23.09
    number      int32 0
    time        datetime64[ns] 2022-05-01
    surface     float64 0.0
    valid_time  (step) datetime64[ns] 2022-05-01T01:00:00 ... 2022-05-02

In [33]:
data1.dom_dir[0,0]

<xarray.DataArray 'dom_dir' ()>
array(1, dtype=int64)
Coordinates:
    latitude   float64 38.22
    longitude  float64 23.09
    number     int32 0
    time       datetime64[ns] 2022-05-01
    surface    float64 0.0